In [1]:
%cd ..

d:\HUST\20232\ML\Project_OCR\HandwritingRecognition


In [2]:
import torch.nn as nn
import torch
from torchsummary import summary

# Model

In [15]:

import torch.nn as nn
import torch

class CRNN(nn.Module):
    def __init__(self, nclass, num_hidden, dropout = 0.1):
        print('>>>> use Crnn-------------\n')
        super(CRNN, self).__init__()

        ks = [ 3,  3,   3,   3,   3,   3,  2]
        ss = [ 1,  1,   1,   1,   1,   1,  1]
        ps = [ 1,  1,   1,   1,   1,   1,  0]
        nm = [32, 64, 128, 128, 256, 256, 256]

        cnn = nn.Sequential()
        def convRelu(i):
            nIn = 1 if i == 0 else nm[i - 1] 
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        # input : (C, H, W) - (1, 32, 512) /800
        convRelu(0)
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d((2, 2)))  # 32, 16, 256/ 400
        convRelu(1)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d((2, 2)))  # 64, 8, 128/ 200
        convRelu(2) 
        convRelu(3) 
        cnn.add_module('pooling{0}'.format(2), nn.MaxPool2d((2, 1)))  # 128, 4, 128/ 200
        convRelu(4)
        convRelu(5) 
        cnn.add_module('pooling{0}'.format(3), nn.MaxPool2d((2, 1)))  # 256, 2, 128/ 200
        convRelu(6)
        self.cnn = cnn
        self.dropout_cnn = nn.Dropout(dropout)
    
        # BiLSTM
        self.biLSTM1 = nn.LSTM(nm[-1], num_hidden, bidirectional=True, batch_first = True)
        self.dropout1 = nn.Dropout(dropout)

        self.biLSTM2 = nn.LSTM(num_hidden*2, num_hidden, bidirectional=True, batch_first = True)
        self.dropout2 = nn.Dropout(dropout)

        # Linear
        self.linear = nn.Linear(num_hidden * 2, nclass)
        
    def forward(self, input):
        # conv features
        x1 = self.cnn(input) # b, 256, 1, 127
        x1 = self.dropout_cnn(x1)
        x1 = torch.squeeze(x1, 2) # b, 256, 127
        x1 = x1.permute(0, 2, 1)  # b, 127, 256

        x2, _  = self.biLSTM1(x1) # b, 127, num_hidden*2
        x2 = self.dropout1(x2) 

        x3, _ = self.biLSTM2(x2) # b, 127, num_hidden*2
        x3 = self.dropout2(x3)

        out = self.linear(x2) # b, 127, num_class

        return out


In [16]:
model = CRNN(151, 128, 0.1)
model

>>>> use Crnn-------------



CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, af

In [14]:
input = torch.rand(64, 1, 32, 768)
output = model(input)
output.shape

torch.Size([64, 191, 151])